In [1]:
import pandas as pd
import numpy as np
import os
import re
from typing import List

# CÁC HÀM XỬ LÝ CHÍNH (CORE FUNCTIONS)

# 1. Hàm chuẩn hóa giá gốc (Original_Price) và giá sau khi giảm (Sales_Price)

In [2]:
def normalize_price_string_logic(price):
    """
    Hàm chuẩn hóa giá, xử lý tất cả giá trị dưới dạng chuỗi (object).
    Phân biệt giữa dấu '.' hàng nghìn và dấu '.' thập phân.
    """
    # 1. Chuyển giá trị đầu vào thành dạng chuỗi để xử lý
    price_str = str(price)

    # 2. Nếu là chuỗi rỗng hoặc 'nan', trả về giá trị rỗng
    if price_str.lower() == 'nan' or not price_str:
        return np.nan
        
    # 3. Xóa các ký tự chữ đi (ví dụ 'Từ ')
    # Giữ lại số và dấu chấm để xử lý ở bước sau
    cleaned_str = re.sub(r'[^\d.]', '', price_str)
    
    if not cleaned_str:
        return np.nan

    # 4. Kiểm tra xem dấu '.' là hàng nghìn hay thập phân
    try:
        # TRƯỜNG HỢP 1: Dấu '.' là ngăn cách hàng nghìn (vd: '450.000', '1.250.000')
        # Quy tắc: Có dấu chấm VÀ phần cuối sau dấu chấm có 3 ký tự
        if '.' in cleaned_str and len(cleaned_str.split('.')[-1]) == 3:
            # Xóa tất cả dấu chấm và chuyển thành số
            final_price = int(cleaned_str.replace('.', ''))
            return final_price
            
        # TRƯỜNG HỢP 2: Dấu '.' là thập phân (vd: '480.0') hoặc không có dấu '.'
        else:
            # Chuyển thành số thực (float), sau đó nhân 1000
            # Việc chuyển thành float sẽ xử lý được cả '480.0' và '280000'
            numeric_val = float(cleaned_str)
            if numeric_val < 1000:
                return int(numeric_val * 1000)
            else:
                return int(numeric_val)

    except (ValueError, IndexError):
        # Nếu có bất kỳ lỗi nào trong quá trình chuyển đổi, trả về rỗng
        return np.nan

# 2. Load và gộp dữ liệu

In [3]:
def load_and_combine_data(file_paths: List[str], column_map: dict) -> pd.DataFrame:
    """
    Đọc nhiều file CSV, chuẩn hóa tên cột cho từng file, loại bỏ cột trùng lặp 
    được tạo ra sau khi chuẩn hóa, rồi gộp chúng lại.
    """
    df_list = []
    for path in file_paths:
        # Bước 1: Đọc từng file
        df_temp = pd.read_csv(path)
        
        # Bước 2: Áp dụng đổi tên cột trước
        df_temp = df_temp.rename(columns=column_map)
        
        # Bước 3: Xóa các cột bị trùng tên (được tạo ra từ Bước 2)
        # Đây là thứ tự đúng để sửa lỗi
        df_temp = df_temp.loc[:, ~df_temp.columns.duplicated(keep='first')]
        
        # Thêm DataFrame đã sạch vào danh sách
        df_list.append(df_temp)
        
    print(f"Đã đọc, chuẩn hóa và dọn dẹp cột cho {len(df_list)} file.")
    
    # Gộp các DataFrame đã hoàn toàn sạch sẽ lại
    return pd.concat(df_list, ignore_index=True)

# 3. Chuẩn hóa tên cột

In [4]:
def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Chuẩn hóa tên các cột và loại bỏ các cột có tên trùng lặp sau khi đổi tên.
    """
    column_mapping = {
        'bus_company': 'bus_name',
        'bus_star': 'bus_rating',
        'bus_type': 'seat_type',
        'duration_h_m': 'duartion',
        'drop_off_point': 'drop_of_point',
        'arrival_date': 'departure_date', 
        'price_original': 'original_price',
        'price_discount': 'sale_price',
        'discount_percent': 'percent_discount',
    }
    
    # Bước 1: Đổi tên các cột theo mapping
    df = df.rename(columns=column_mapping)
    
    # Bước 2: Xóa các cột bị trùng tên, chỉ giữ lại cột xuất hiện đầu tiên
    # Đây là dòng code quan trọng để sửa lỗi
    df = df.loc[:, ~df.columns.duplicated(keep='first')]
    
    print("Đã chuẩn hóa và loại bỏ các cột có tên trùng lặp.")
    return df

# 4. Áp dụng hàm chuẩn hóa giá cả

In [5]:
def clean_price_columns(df: pd.DataFrame) -> pd.DataFrame:
    # Áp dụng hàm mới, tạo cột 'price_cleaned' và 'sale_price_cleaned'
    df['price_cleaned'] = df['original_price'].apply(normalize_price_string_logic)
    df['sale_price_cleaned'] = df['sale_price'].apply(normalize_price_string_logic)
    # In kết quả ra để so sánh
    print("Kết quả sau khi làm sạch (xử lý tất cả như chuỗi):")
    print(df[['original_price', 'price_cleaned']].head(-10))
    print(df['price_cleaned'].dtype)
    print("======================================================")
    print("Kết quả sau khi làm sạch (xử lý tất cả như chuỗi):")
    print(df[['sale_price', 'sale_price_cleaned']].head(-10))
    print(df['sale_price_cleaned'].dtype)
    # Ghi đè cột
    df['original_price'] = df['price_cleaned']
    df['sale_price'] = df['sale_price_cleaned']
    print("Đã ghi đè cột 'original_price' bằng dữ liệu từ 'price_cleaned'")
    print("Đã ghi đè cột 'sale_price' bằng dữ liệu từ 'sale_price_cleaned'")
    # Xóa cột 'price_cleaned' và 'sale_price_cleaned' không còn cần thiết
    df = df.drop('price_cleaned', axis=1)
    df = df.drop('sale_price_cleaned', axis=1)
    print("Đã xóa cột 'price_cleaned'")
    print("Đã xóa cột 'sale_price_cleaned'")

    return df


# 5. Hàm xử lí giá trị bị thiếu

In [6]:
def handle_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Xử lý các giá trị thiếu: xóa cột không cần thiết và điền giá trị còn lại.
    """
    # 1. Xóa các cột có quá nhiều giá trị thiếu hoặc không cần thiết
    cols_to_drop = [col for col in df.columns if 'Unnamed' in col]
    # Dựa theo notebook, các cột này cũng được xóa
    cols_to_drop.extend(['start_point', 'destination'])
    # Xóa cột arrival_date vì nó bị trùng lặp thông tin hoặc không đồng nhất
    if 'arrival_date' in df.columns:
        cols_to_drop.append('arrival_date')
        
    df = df.drop(columns=cols_to_drop, errors='ignore')
    print(f"Đã xóa các cột không cần thiết: {cols_to_drop}")

    # 2. Điền giá trị thiếu cho các cột rating
    rating_cols = ['An toàn', 'Thông tin chính xác', 'Thông tin đầy đủ',
                   'Thái độ nhân viên', 'Tiện nghi & thoải mái',
                   'Chất lượng dịch vụ', 'Đúng giờ']
    
    # Chỉ xử lý các cột rating tồn tại trong df
    existing_rating_cols = [col for col in rating_cols if col in df.columns]
    df[existing_rating_cols] = df.groupby('bus_name')[existing_rating_cols].transform(lambda x: x.ffill().bfill())
    # Nếu sau đó vẫn còn NaN (do cả nhà xe không có rating nào), có thể fill bằng median hoặc 0
    df[existing_rating_cols] = df[existing_rating_cols].fillna(df[existing_rating_cols].median())
    print("Đã điền giá trị thiếu cho các cột rating.")

    # 3. Điền giá trị thiếu cho các cột dạng object (chuỗi)
    object_cols = df.select_dtypes(include=['object']).columns
    df[object_cols] = df[object_cols].fillna('')
    print("Đã điền giá trị thiếu cho các cột dạng chuỗi.")
    
    return df

# 6. Hàm lưu DataFrame đã làm sạch vào CSV

In [7]:
def save_cleaned_data(df: pd.DataFrame, output_path: str):
    """
    Lưu DataFrame đã làm sạch vào file CSV.
    """
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Đã tạo thư mục: {output_dir}")

    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"File đã được lưu thành công tại: {output_path}")

# HÀM THỰC THI QUY TRÌNH (PIPELINE EXECUTION FUNCTION)

In [8]:
def run_cleaning_pipeline(file_paths: List[str], output_path: str) -> pd.DataFrame:
    """
    Thực thi toàn bộ quy trình làm sạch dữ liệu.
    """
    print("===== BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU =====")
    
    # Định nghĩa column mapping ở đây
    column_mapping = {
        'bus_company': 'bus_name',
        'bus_star': 'bus_rating',
        'bus_type': 'seat_type',
        'duration_h_m': 'duration',
        'drop_off_point': 'drop_of_point',
        'arrival_date': 'departure_date', 
        'price_original': 'original_price',
        'price_discount': 'sale_price',
        'discount_percent': 'percent_discount',
    }
    
    # Bước 1: Đọc và gộp dữ liệu (với hàm đã được cải tiến)
    df = load_and_combine_data(file_paths, column_map=column_mapping)
    
    # Bước 2: Chuẩn hóa tên cột (giờ chỉ còn nhiệm vụ xóa cột trùng lặp nếu có)
    df = standardize_columns(df)
    
    # Bước 3: Xóa dữ liệu trùng lặp
    initial_rows = len(df)
    df = df.drop_duplicates().reset_index(drop=True)
    print(f"Đã xóa {initial_rows - len(df)} dòng trùng lặp.")
    
    # Bước 4: Làm sạch cột giá
    df = clean_price_columns(df)
    
    # Bước 5: Xử lý giá trị thiếu
    df = handle_missing_values(df)
    
    # Bước 6: Lưu kết quả
    save_cleaned_data(df, output_path)
    
    print("===== KẾT THÚC QUY TRÌNH LÀM SẠCH DỮ LIỆU =====")
    return df

In [9]:
if __name__ == '__main__':
    # Định nghĩa đường dẫn tới các file dữ liệu thô
    # Giả sử file code này được đặt cùng cấp với thư mục 'data'
    raw_data_dir = '../../data/raw'
    files_to_process = [
        os.path.join(raw_data_dir, 'sg_nt_11102025.csv'),
        os.path.join(raw_data_dir, 'hn_th_15102025.csv'),
        os.path.join(raw_data_dir, 'Sài Gòn-Gia Lai-17-10-2025.csv')
    ]
    
    # Định nghĩa đường dẫn file đầu ra
    processed_data_path = '../../data/processed/cleaned_bus_data.csv'
    
    # Chạy quy trình
    cleaned_df = run_cleaning_pipeline(
        file_paths=files_to_process,
        output_path=processed_data_path
    )
    
    # Hiển thị thông tin của DataFrame đã làm sạch
    print("\\n===== THÔNG TIN DỮ LIỆU SAU KHI LÀM SẠCH =====")
    cleaned_df.info()

===== BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU =====
Đã đọc, chuẩn hóa và dọn dẹp cột cho 3 file.
Đã chuẩn hóa và loại bỏ các cột có tên trùng lặp.
Đã xóa 2 dòng trùng lặp.
Kết quả sau khi làm sạch (xử lý tất cả như chuỗi):
     original_price  price_cleaned
0           450.000       450000.0
1           350.000       350000.0
2           280.000       280000.0
3           299.000       299000.0
4           480.000       480000.0
...             ...            ...
1003          500.0       500000.0
1004          350.0       350000.0
1005          350.0       350000.0
1006          650.0       650000.0
1007          350.0       350000.0

[1008 rows x 2 columns]
float64
Kết quả sau khi làm sạch (xử lý tất cả như chuỗi):
     sale_price  sale_price_cleaned
0       320.000            320000.0
1       300.000            300000.0
2       250.000            250000.0
3       199.000            199000.0
4       400.000            400000.0
...         ...                 ...
1003      450.0           